## Module 1 Homework

## Docker & SQL

In this homework we'll prepare the environment 
and practice with Docker and SQL


## Question 1. Knowing docker tags

Run the command to get information on Docker 

```docker --help```

Now run the command to get help on the "docker build" command:

```docker build --help```

Do the same for "docker run".

Which tag has the following text? - *Automatically remove the container when it exits* 

- `--delete`
- `--rc`
- `--rmc`
- `--rm`

**Solution:**

`--rm` Automatically removes the container when it exits


## Question 2. Understanding docker first run 

Run docker with the python:3.9 image in an interactive mode and the entrypoint of bash.
Now check the python modules that are installed ( use ```pip list``` ). 

What is version of the package *wheel* ?

- 0.42.0
- 1.0.0
- 23.0.1
- 58.1.0

**Solution:**

To create the docker container, the following command is used:
```sh
docker run -it --entrypoint=bash python:3.9
```
To get the version of the python-package `wheel` the following command is used:
```sh
pip list | grep wheel
``` 
This results in `wheel 0.42.0`

# Prepare Postgres

Run Postgres and load data as shown in the videos
We'll use the green taxi trips from September 2019:

```wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz```

You will also need the dataset with zones:

```wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv```

Download this data and put it into Postgres (with jupyter notebooks or with a pipeline)

For this purpose 2 container are created and run with docker-compose
- Container with `pgAdmin4`
- Container with `postgresql` database

```bash
URL=https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz

python ingest_data.py \
  --user=root \
  --password=root \
  --host=localhost \
  --port=5432 \
  --db=ny_taxi \
  --table_name=green_taxi_trips \
  --url=${URL}

URL=https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

python ingest_data.py \
  --user=root \
  --password=root \
  --host=localhost \
  --port=5432 \
  --db=ny_taxi \
  --table_name=zones \
  --url=${URL}

```

Done!

## Question 3. Count records 

How many taxi trips were totally made on September 18th 2019?

Tip: started and finished on 2019-09-18. 

Remember that `lpep_pickup_datetime` and `lpep_dropoff_datetime` columns are in the format timestamp (date and hour+min+sec) and not in date.

- 15767
- 15612
- 15859
- 89009

**Solution**:

*Query:*
```sql
SELECT
	COUNT(*)
FROM
	green_taxi_trips
WHERE
	DATE(lpep_pickup_datetime) = '2019-09-18' AND
	DATE(lpep_dropoff_datetime) = '2019-09-18';
```

*Result*: 15612

## Question 4. Largest trip for each day

Which was the pick up day with the largest trip distance
Use the pick up time for your calculations.

- 2019-09-18
- 2019-09-16
- 2019-09-26
- 2019-09-21

**Solution**:

*Query*:
```sql
SELECT
	DATE(lpep_pickup_datetime)
FROM
	green_taxi_trips
ORDER BY
	trip_distance DESC
LIMIT
	1;
```

*Result*:
- **`date`**: `2019-09-26`
- **`trip_distance`**: `341.64`

## Question 5. The number of passengers

Consider `lpep_pickup_datetime` in '2019-09-18' and ignoring Borough has `Unknown`

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?
 
- "Brooklyn" "Manhattan" "Queens"
- "Bronx" "Brooklyn" "Manhattan"
- "Bronx" "Manhattan" "Queens" 
- "Brooklyn" "Queens" "Staten Island"


**Solution**:

*Query*:
```sql
SELECT
	z."Borough",  -- Name of Borough
	SUM(t.total_amount) AS borough_amount -- sum amount of Borough-group
FROM
	green_taxi_trips AS t
JOIN zones AS z	-- Joining the taxi and zones table (to obtain Names)
	ON t."PULocationID" = z."LocationID"
WHERE
	DATE(t.lpep_pickup_datetime) = '2019-09-18' AND  -- at given date
	z."Borough" != 'Unknown'  -- removing "Unknown"
GROUP BY   -- Grouping by Borough
	z."Borough"
HAVING 	   -- Only use grouped values of amounts over 50k  
	SUM(t.total_amount) > 50000
ORDER BY   -- Sort by sum amount in Borough descending
	borough_amount DESC
LIMIT 3;
```

Output:

|   | **`Borough`**<br> (text) | **`borough_amount`**<br> (double)    |
| - | ------------------------ | ------------------------------------ |
| 1 | Brooklyn				   | 96333.24							  |
| 2 | Manhattan 			   | 92271.30							  |
| 3 | Queens				   | 78671.71							  |

## Question 6. Largest tip

For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip?
We want the name of the zone, not the id.

Note: it's not a typo, it's `tip` , not `trip`

- Central Park
- Jamaica
- JFK Airport
- Long Island City/Queens Plaza


**Solution**:

*Query*:
```sql
-- For the passengers picked up in September 2019 in the zone name Astoria 
-- which was the drop off zone that had the largest tip?

SELECT
	zdo."Zone" AS dropoff_zone,
	MAX(t.tip_amount) AS max_zone_tip
FROM
	green_taxi_trips AS t
JOIN zones AS zpu
	ON t."PULocationID" = zpu."LocationID"
JOIN zones AS zdo
	ON t."DOLocationID" = zdo."LocationID"
WHERE
	zpu."Zone" = 'Astoria' AND
	TO_CHAR(lpep_pickup_datetime, 'YYYY-MM') = '2019-09' 
GROUP BY
	dropoff_zone
ORDER BY
	max_zone_tip DESC
LIMIT
	1;
```
*Result*:
- **`dropoff_zone`**: JFK Airport
- **`max_zone_tip`**: 62.31


## Terraform

In this section homework we'll prepare the environment by creating resources in GCP with Terraform.

In your VM on GCP/Laptop/GitHub Codespace install Terraform. 
Copy the files from the course repo
[here](https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main/01-docker-terraform/1_terraform_gcp/terraform) to your VM/Laptop/GitHub Codespace.

Modify the files as necessary to create a GCP Bucket and Big Query Dataset.


## Question 7. Creating Resources

After updating the main.tf and variable.tf files run:

```
terraform apply
```

Paste the output of this command into the homework submission form.


**Solution**:
The terraform-files used for this task can be found in the sub-directory [terraform/](terraform/). To avoid putting the gcp-credentials or the local path to the credentials into the terraform-files, an environment variable is used.

```bash
export TF_VAR_credentials=path/to/gcp-credentials.json
```
This specific type of environment-variable `TF_VAR_<name-of-variable>` has the property that Terraform uses them for the value specified, without providing them in the `variables.tf` file. The default value for the credentials in the [variables.tf](terraform/variables.tf) is left blank and overwritten by `TF_VAR_credentials`.

```yaml
Terraform used the selected providers to generate the following execution plan. Resource actions are indicated with the
following symbols:
  + create

Terraform will perform the following actions:

  # google_bigquery_dataset.Homework1_dataset will be created
  + resource "google_bigquery_dataset" "Homework1_dataset" {
      + creation_time              = (known after apply)
      + dataset_id                 = "homework1_dataset_410018"
      + default_collation          = (known after apply)
      + delete_contents_on_destroy = false
      + effective_labels           = (known after apply)
      + etag                       = (known after apply)
      + id                         = (known after apply)
      + is_case_insensitive        = (known after apply)
      + last_modified_time         = (known after apply)
      + location                   = "EU"
      + max_time_travel_hours      = (known after apply)
      + project                    = "dtc-de-410018"
      + self_link                  = (known after apply)
      + storage_billing_model      = (known after apply)
      + terraform_labels           = (known after apply)
    }

  # google_storage_bucket.Homework1-bucket will be created
  + resource "google_storage_bucket" "Homework1-bucket" {
      + effective_labels            = (known after apply)
      + force_destroy               = true
      + id                          = (known after apply)
      + location                    = "EU"
      + name                        = "bucket-410018"
      + project                     = (known after apply)
      + public_access_prevention    = (known after apply)
      + self_link                   = (known after apply)
      + storage_class               = "STANDARD"
      + terraform_labels            = (known after apply)
      + uniform_bucket_level_access = (known after apply)
      + url                         = (known after apply)

      + lifecycle_rule {
          + action {
              + type = "AbortIncompleteMultipartUpload"
            }
          + condition {
              + age                   = 1
              + matches_prefix        = []
              + matches_storage_class = []
              + matches_suffix        = []
              + with_state            = (known after apply)
            }
        }
    }

Plan: 2 to add, 0 to change, 0 to destroy.
```